In [5]:
from pyspark.sql.functions import col, desc, asc, when
import pyspark.sql.functions as F
import pygeohash as pgh
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd

geohashEncodeUDF = F.udf(lambda x, y: pgh.encode(x, y))
geohashDecodeUDF = F.udf(lambda s: pgh.decode(s))

### Strangely Snowy: Find a location that contains snow while its surroundings do not. Why does this occur? Is it a high mountain peak in a desert?
I originally started by looking for all the points in the dataset on Dec 26th, 2016 that had snow and realized there were too many points to look at them all individually. So I looked up a map that let me pick a point and get the lat and long values for the point and picked a spot in Arizona because some areas have snow in winter, but that didn't give me many results so I started looking in the CA/NV border because of the mountain ranges in the area.

I found an area in the Stanislaus National Forest between December 26-29, 2016 that had only one point in a bounding region with a snow depth higher than 0. The area's bounding latitude values were between 37.3-37.8, and its bounding longitude values were -121 and -120. 
The coordinates of the snowy point are 38.021557396033124 lat and -120.00482559776982 long.
The elevation of this point is 4996.6 ft, so it makes sense that the surrounding points with lower elevation would have no snow on them.

In [ ]:
df = spark.read.load('hdfs://orion01:25001/namanl_218_20151010_0000_003.grb.tdv.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

In [ ]:
df.count()

In [ ]:
df.take(1)

In [ ]:
snowy = df.filter(df.snow_depth_surface > 0)
snowy.take(4)

In [ ]:
snowy.count()

In [ ]:
df.createOrReplaceTempView("total")

arizona = spark.sql('SELECT * FROM total WHERE 2_lat > 32 AND 2_lat < 34 AND 3_lon > -111 AND 3_lon < -109 AND snow_depth_surface == 0')

# around = snowy.filter(df.2_lat >= 32 && df.2_lat >= 34).filter(df.3_lon >= -109 && df.3_lon >= -111)

In [ ]:
arizona.count()

In [ ]:
arizona.take(5)

In [ ]:
diablo = spark.sql('SELECT * FROM total WHERE 2_lat > 35 AND 2_lat < 38 AND 3_lon > 120 AND 3_lon < 123 AND snow_depth_surface > 0')
diablo.count()

In [ ]:
#  namanl_218_20161227_1200_003.grb.tdv.gz
dec27 = spark.read.load('hdfs://orion01:25001/namanl_218_201612*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

In [ ]:
dec27.createOrReplaceTempView("december")

wovoka_snow = spark.sql('SELECT * FROM december WHERE 2_lat > 37.5 AND 2_lat < 38.5 AND 3_lon < -118 AND 3_lon > -121 AND snow_depth_surface > 0')
wovoka_dry = spark.sql('SELECT * FROM december WHERE 2_lat > 37.5 AND 2_lat < 38.5 AND 3_lon < -118 AND 3_lon > -121 AND snow_depth_surface = 0')

wovoka = spark.sql('SELECT 2_lat, 3_lon, snow_depth_surface FROM december WHERE 2_lat > 37.5 AND 2_lat < 38.5 AND 3_lon < -118 AND 3_lon > -121')


In [ ]:
wovoka_snow.count()

In [ ]:
wovoka_dry.count()

In [ ]:
wovoka.count()

In [ ]:
wovoka.take(10)

In [ ]:
wovoka.take(75)

In [ ]:
dec27.createOrReplaceTempView("wovoka")

bound = spark.sql('SELECT 2_lat, 3_lon, snow_depth_surface FROM wovoka WHERE 37.3 < 2_lat AND 2_lat < 38.1 AND -121 < 3_lon AND 3_lon < -120')


In [ ]:
bound.count()

In [ ]:
bound.take(168)

#### Climate Chart: Given a Geohash prefix as an input, build a function that will create a climate chart for the region. This includes high, low, and average temperatures, as well as monthly average rainfall (precipitation).

To solve this problem, I imported a few days worth of data from every month in 2016 so that any random fluctuations in weather by week would be accounted for. I then added a column of geohashes so that I could select only the points in each month's dataframes that started with the 2 character geohash, as well as renamed the lat and lon columns to make calculating the geohashes go more smoothly.

My next step was to make a function that would get the necessary data from each of the month dataframes, average them, and then store them in another dataframe that could be saved into a file. This file is read by the chart function that was linked in the project spec, though I did modify it slightly so that it treated the temperatures as though they were in Fahrenheit by default. The charting method then reads teh file and plots teh high, low, and average temperatures per month, as well as the average rainfall per month in inches (the numbers are pretty low, but I think I just chose days that didn't have as much rain...)

In [52]:
jan = spark.read.load('hdfs://orion01:25001/namanl_218_201601*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

feb = spark.read.load('hdfs://orion01:25001/namanl_218_201602*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

mar = spark.read.load('hdfs://orion01:25001/namanl_218_201603*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

apr = spark.read.load('hdfs://orion01:25001/namanl_218_201604*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

may = spark.read.load('hdfs://orion01:25001/namanl_218_201605*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

jun = spark.read.load('hdfs://orion01:25001/namanl_218_201606*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

In [53]:
jul = spark.read.load('hdfs://orion01:25001/namanl_218_201607*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

aug = spark.read.load('hdfs://orion01:25001/namanl_218_201608*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

sep = spark.read.load('hdfs://orion01:25001/namanl_218_201609*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

octo = spark.read.load('hdfs://orion01:25001/namanl_218_201610*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

nov = spark.read.load('hdfs://orion01:25001/namanl_218_201611*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

dec = spark.read.load('hdfs://orion01:25001/namanl_218_201612*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

In [54]:


jan = jan.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
jan = jan.withColumn("geohash", geohashEncodeUDF(jan.lat, jan.lon))

feb = feb.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
feb = feb.withColumn("geohash", geohashEncodeUDF(feb.lat, feb.lon))

mar = mar.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
mar = mar.withColumn("geohash", geohashEncodeUDF(mar.lat, mar.lon))

apr = apr.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
apr = apr.withColumn("geohash", geohashEncodeUDF(apr.lat, apr.lon))

may = may.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
may = may.withColumn("geohash", geohashEncodeUDF(may.lat, may.lon))

jun = jun.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
jun = jun.withColumn("geohash", geohashEncodeUDF(jun.lat, jun.lon))

jul = jul.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
jul = jul.withColumn("geohash", geohashEncodeUDF(jul.lat, jul.lon))

aug = aug.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
aug = aug.withColumn("geohash", geohashEncodeUDF(aug.lat, aug.lon))

sep = sep.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
sep = sep.withColumn("geohash", geohashEncodeUDF(sep.lat, sep.lon))

octo = octo.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
octo = octo.withColumn("geohash", geohashEncodeUDF(octo.lat, octo.lon))

nov = nov.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
nov = nov.withColumn("geohash", geohashEncodeUDF(nov.lat, nov.lon))

dec = dec.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
dec = dec.withColumn("geohash", geohashEncodeUDF(dec.lat, dec.lon))
    
jan.take(1)

[Row(time=1452697200000, lat=37.03410173798036, lon=-86.05695551596708, albedo_surface=15.25, precipitable_water_entire_atmosphere_single_layer='null', pressure_maximum_wind=19526.98, pressure_surface=100037.0, pressure_tropopause=21726.576, relative_humidity_zerodegc_isotherm=66.0, snow_depth_surface=0.0, temperature_surface=270.7961, temperature_tropopause=213.32837, total_cloud_cover_entire_atmosphere_single_layer='null', total_precipitation_surface_3_hour_accumulation=0.0, vegetation_surface=30.25, visibility_surface=24229.455, wilting_point_surface=0.083749995, wind_speed_gust_surface='null', _c18=None, geohash='dndf9tz5r8gj')]

In [ ]:
def climate_chart(months, geohash):
    avgs = pd.DataFrame()

    avgs['month'] = None
    avgs['h_temp'] = None
    avgs['l_temp'] = None
    avgs['avg_precip'] = None
    avgs['avg_temp'] = None
    
    
    for i in range(0, len(months)):
        min_temp = months[i].select(min(when(months[i].geohash.startswith(geohash), months[i].temperature_surface)).alias("min_temp"))
        max_temp = months[i].select(max(when(months[i].geohash.startswith(geohash), months[i].temperature_surface)).alias("max_temp"))
        avg_temp = months[i].select(avg(when(months[i].geohash.startswith(geohash), months[i].temperature_surface)).alias("avg_temp"))
        avg_rain = months[i].select(avg(when(months[i].geohash.startswith(geohash), months[i].total_precipitation_surface_3_hour_accumulation)).alias("avg_rain"))

        # <month-num>  <high-temp>  <low-temp>  <avg-precip>  <avg-temp>
        data = [i+1, k2f(max_temp.head().max_temp), k2f(float(min_temp.head().min_temp)), \
                   avg_rain.head().avg_rain, k2f(avg_temp.head().avg_temp)]
        
        avgs.loc[len(avgs.index)] = data
        
    return avgs  

In [66]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
import matplotlib
from matplotlib import rc

def c2f(t):
    return (t*9/5.0)+32

def k2c(t):
    return t-273.15

def k2f(t):
    return (t*9/5.0)-459.67

#------------------------------------------------------------------------------------#

def climate_map(filename):
    with open(filename, 'r') as f:
        first_line = f.readline()[1:].strip()
        lines = (line for line in f if not line.startswith(','))
        data = np.loadtxt(lines, delimiter=' ')
        
    data = data[:, 1:]
    
    data[:, 0] = data[:, 0] - 1
    data[:, 3] = data[:, 3] * 0.0393701

    plt.ion()
    plt.clf()
    fig = plt.figure(1)
    fig.subplots_adjust(hspace=.20)
    gs = gridspec.GridSpec(2, 1, height_ratios=[1.75, 1])
    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1], sharex=ax0)
    plt.setp(ax0.get_xticklabels(), visible=False) # disable upper axis label

    ax0.patch.set_facecolor('None')
    ax1.patch.set_facecolor('None')
    
    title = "Climate Overview"

    plt.suptitle(title, fontsize=14)

    y = 0
    ax0.plot([0, data[:, 0].max() + 1], [y, y], zorder=-1, color='#888888',
                alpha=.75, dashes=(8, 2))

    rects0 = ax0.bar(.35 + data[:, 0], data[:, 2] - data[:, 1], bottom=data[:, 1],
            width=.6, color='#df3c3c', edgecolor='#731515')

    rects1 = ax1.bar(.35 + data[:, 0], data[:, 3], color='#1b7edb', width=.6,
            edgecolor='#1d4871')
    
    print(data[:,3])

    plt.xticks(np.arange(0,12) + .4, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        rotation=30)

    ax0.spines['left'].set_visible(True)
    ax1.spines['left'].set_visible(True)

    for tic in ax0.xaxis.get_major_ticks():
        tic.tick1On = tic.tick2On = False

    for tic in ax0.yaxis.get_major_ticks():
        tic.tick2On = False

    for tic in ax1.xaxis.get_major_ticks():
        tic.tick1On = tic.tick2On = False

    for tic in ax1.yaxis.get_major_ticks():
        tic.tick2On = False

    for rect in rects1:
        height = rect.get_height()
        ax1.text(rect.get_x() + rect.get_width()/2., 1.08*height,
            '%.2f' % (height), ha='center', va='bottom', color='#1d4871')

    for r, rect in enumerate(rects0):
        height = rect.get_height()
        print(height)
        ax0.text(rect.get_x() + rect.get_width()/2., rect.get_y() + 1.08*height,
            '%d' % int(height + rect.get_y()), ha='center', va='bottom',
            color='#731515')
        ax0.text(rect.get_x() + rect.get_width()/2., rect.get_y() - 2,
            '%d' % int(rect.get_y()), ha='center', va='top', color='#731515')
        ax0.plot([rect.get_x() + .05, rect.get_x() + rect.get_width() - .05],
                [data[r, 4], data[r, 4]], color='#731515')

    ax0.set_ylabel('Temperature (F)')
    ax1.set_ylabel('Precipitation (in)')

    plt.savefig('climate.pdf', bbox_inches='tight')

In [ ]:
months = [jan, feb, mar, apr, may, jun, jul, aug, sep, octo, nov, dec]

avgs = climate_chart(months=months, geohash='dj')

In [ ]:
avgs.head(12)
# print(len(avgs))

In [ ]:
avgs.to_csv('avgs.csv',sep=' ')

In [ ]:
climate = climate_map('avgs.csv')

# plt.show()

#### Fogless Mansion: After becoming rich from your startup, you are looking for the perfect location to build your Bay Area mansion with unobstructed views. Find the locations that are the least foggy and show them on a map.

My first step to solve this problem was to import all of the plotting libraries from matplotlib and pandas. I then renamed the columns for latitude and longitude so that I wouldn't get errors from having ints in the column names when referencing them. I found the general latitude and longitude box for the bay area and started filtering the dataset with those values. 

I chose to have the ```visibility_surface``` of the point be higher than 24000m for the best visibility possible, and I found about 4400 points that way. Sadly, this set of results was not very good and when plotted gave really weird results with arcing lines of high visibility, so I scrapped it and adjusted my filter parameters.

My second test added a few more files from the summer of 2016 when the fog levels would be highest between June and August rather than just July in the first time around.

In [ ]:
df = df.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")


In [ ]:
df.take(1)

In [ ]:
df.createOrReplaceTempView("df")

clear = spark.sql('SELECT lat, lon, visibility_surface FROM df WHERE visibility_surface > 24000 \
                    AND 38 > lat AND lat > 37 AND 122 > lon AND 120 > 37')



In [ ]:
clear.count()

In [ ]:
clear = clear.sort("lon")

In [ ]:
clear.take(10)

In [ ]:
clear_df = clear.toPandas()

clear_df.plot(x="lon", y="lat", kind="scatter", c="visibility_surface",
        colormap="YlOrRd")

In [ ]:
summer = spark.read.load('hdfs://orion01:25001/namanl_218_20160*0600*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

In [ ]:
summer = summer.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")

summer.createOrReplaceTempView("summer")

summ = spark.sql('SELECT lat, lon, visibility_surface FROM summer WHERE 24000 > visibility_surface \
                    AND visibility_surface > 19000 AND 40 > lat AND lat > 36 AND -120 > lon AND lon > -125')

In [ ]:
summ.count()

In [ ]:
summ_df = summ.toPandas()


In [ ]:
summ_df = summ_df.sort_values(by='lon', ascending=True)

summ_df.head()


In [ ]:
summ.take(10)

In [ ]:
summ_df.plot(x="lon", y="lat", kind="scatter", c="visibility_surface",
        colormap="YlOrRd")

In [ ]:
worldmap = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
worldmap[worldmap["name"] == "United States of America"].plot(color="lightgrey")

x = summ_df['lon']
y = summ_df['lat']

plt.scatter(x, y, s=z/20000, c=z, cmap='RdPu')

plt.colorbar(label='visibility_surface')

plt.xlim([-128, -118])
plt.ylim([35, 42.5])

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

### SolarWind, Inc.: 
#### After getting rich from your travel startup you get bored and start a new company; here, you want to help power companies plan out the locations of solar and wind farms across North America. Locate the top 3 places for solar and wind farms, as well as a combination of both (solar + wind farm). You will report a total of 9 Geohashes as well as their relevant attributes (for example, cloud cover and wind speeds).

For this problem, I stored all the files at 12pm for the entire 2019 data folder because there are valid `wind_speed_gust_surface` and `total_cloud_cover_entire_atmosphere_single_layer` values for 2019. I had to abandon my favorite 2016 dataset :'(

The next thing I did was try a few different ranges of wind gust speeds to filter on in North America, but ultimately decided on 50-70mph so that it was still a somewhat common speed. Some geohashes that were the best for wind farms based on these critera are *9wm6619rmzpp* (69.85mph), *9vqengzgyf40* (69.91mph), and *9xhukcmqdyt8* (69.87mph), all of which were not in the middle of the ocean (because that was a susprisingly annoying problem I kept running into).

I followed the same steps for cloud coverage (low cloud coverage means high sun exposure for solar panels) and eventually settled on a cloud coverage range of 0-15%. This was a bit harder to settle on because of duplicate points from all the files, but to fix that I aggreggated the average cloud coverage of all the points by geohash, this way I was sure that there was only one entry per location, and it could give a baseline estimate for yearly sun exposure. Three geohashes with low cloud coverage that could have solar energy farms are *djv29jbe3xt4* (0%), *dryzdfv0h5ke* (0%), and *9qn4n86pqg6g* (0%).

To find locations where combo wind and solar farms could be built, I combined both of the matching criteria for wind and solar farms into one dataset from 2019. From here i found another 3 geohashes that would be suitable: *9xmfjt08b78u* (50.03mph wind, 0% cloud cover), *9xqjvg1hwpmu* (50.03mph, 0%), and *9wht5bzj7k0v* (50.04mph, 0%).

In [2]:
df = spark.read.load('hdfs://orion01:25001/2019/namanl_218_2019*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

df = df.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
df = df.withColumn("geohash", geohashEncodeUDF(df.lat, df.lon))

In [ ]:
# df = df.withColumn("wind_speed", df.pressure_maximum_wind * 0.00256) # convert wind pressure to wind speed (mph)
df = df.withColumn("wind_speed_gust_surface", df.wind_speed_gust_surface * 2.23694) # convert gust speed to mph



In [ ]:
# df = df.sort("total_cloud_cover_entire_atmosphere_single_layer")

# df = df.sort("albedo_surface")

df.take(1)

In [ ]:
df.createOrReplaceTempView("farms")


# wind.count()


In [ ]:
wind = spark.sql('SELECT * FROM farms WHERE 70 > wind_speed_gust_surface AND wind_speed_gust_surface > 50 \
AND 24 < lat AND lat < 50 AND -66 > lon AND lon > -124')


In [ ]:
wind.count()

In [ ]:
hashes = wind.groupBy("geohash")\
            .agg(F.avg("wind_speed_gust_surface").alias("wind_speed"), \
                F.avg("lat").alias("lat"), F.avg("lon").alias("lon"))

# 

In [ ]:
wind_df = hashes.toPandas()

In [ ]:
wind_df.count()

In [ ]:
worldmap = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
worldmap[worldmap["name"] == "United States of America"].plot(color="lightgrey")

# Plotting our Impact Energy data with a color map
x = wind_df['lon']
y = wind_df['lat']
z = wind_df['wind_speed']
plt.scatter(x, y, s=z/1000, c=z, cmap='Blues')

plt.colorbar(label='Maximum Wind Gust Speed (mph)')

plt.xlim([-128, -65])
plt.ylim([24, 51])

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
sun = spark.sql('SELECT * FROM farms WHERE total_cloud_cover_entire_atmosphere_single_layer < 15 AND \
        24 < lat AND lat < 50 AND -66 > lon AND lon > -124')

sun = sun.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
sun = sun.withColumn("geohash", geohashEncodeUDF(df.lat, df.lon))

sun.count()
# 60 < albedo_surface AND \

In [ ]:
hashes = sun.groupBy("geohash").agg(\
                                    F.avg("total_cloud_cover_entire_atmosphere_single_layer").alias("cloud_cover"), \
                                    F.avg("lat").alias("lat"), F.avg("lon").alias("lon"))

hashes.limit(5).show()

In [ ]:
sun_df = hashes.toPandas()

sun_df.count()

In [ ]:
worldmap = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
worldmap[worldmap["name"] == "United States of America"].plot(color="lightgrey")

# Plotting our Impact Energy data with a color map
x = sun_df['lon']
y = sun_df['lat']
z = sun_df['cloud_cover']
plt.scatter(x, y, s=z/25, c=z, cmap='YlOrRd')

plt.colorbar(label='Cloud Cover (%)')

plt.xlim([-128, -65])
plt.ylim([24, 51])

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
farms = spark.sql('SELECT * FROM farms WHERE total_cloud_cover_entire_atmosphere_single_layer < 15 AND \
        70 > wind_speed_gust_surface AND wind_speed_gust_surface > 50 AND \
        24 < lat AND lat < 50 AND -66 > lon AND lon > -124')

farms = farms.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
farms = farms.withColumn("geohash", geohashEncodeUDF(farms.lat, farms.lon))


In [ ]:
farms.count()

In [ ]:
farms_df = farms.toPandas()

In [ ]:
worldmap = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
worldmap[worldmap["name"] == "United States of America"].plot(color="lightgrey")

# Plotting our Impact Energy data with a color map
x = farms_df['lon']
y = farms_df['lat']
z = farms_df['total_cloud_cover_entire_atmosphere_single_layer']
# z = farms_df['wind_speed_gust_surface']

plt.scatter(x, y, s=z/25, c=z, cmap='Greens')

plt.colorbar(label='Cloud Cover (%)')

plt.xlim([-128, -65])
plt.ylim([24, 51])

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.scatter(farms_df.lon, farms_df.lat, c="Orange", \
           s=farms_df.total_cloud_cover_entire_atmosphere_single_layer, label="Cloud Cover", \
           alpha=0.3, edgecolors='none')

ax.scatter(farms_df.lon, farms_df.lat, c="Blue", \
           s=farms_df.wind_speed_gust_surface/40, label="Wind Speed", \
           alpha=0.3, edgecolors='none')

ax.legend()

plt.show()

In [ ]:
wind.createOrReplaceTempView("wind")
sun.createOrReplaceTempView("sun")

In [ ]:
wind = wind.sort(desc("wind_speed_gust_surface"))

In [ ]:
wind.take(50)

In [ ]:
sun = sun.sort("total_cloud_cover_entire_atmosphere_single_layer")

In [ ]:
sun.take(50)

In [ ]:
farms = farms.sort("total_cloud_cover_entire_atmosphere_single_layer", "wind_speed_gust_surface")

In [ ]:
farms.take(50)

#### Climate Change: Using two-character geohash aggregates across the entire NAM grid, determine temperature trends over the past 5 years. With the regions that have experienced an increase in temperatures, build a correlation matrix using Pearson’s correlation coefficient (PCC) to determine how the variables influence one another. Finally, determine whether or not the correlations are different based on the region (e.g., maybe temperature has increased in lockstep with humidity in one location but not another). Analyze your results: can you draw any conclusions from what you’ve found?

#### Weather Station: Write a multi-threaded server (outside of Spark) that reads files from the dataset — one file per thread — and then streams them out on a socket for a Spark streaming context to consume (note: not ALL the files have to be opened at once! :-)). The program should produce records as fast as the network will support, i.e., faster than real time. 

####

In [ ]:
# For a new value newValue, compute the new count, new mean, the new M2.
# mean accumulates the mean of the entire dataset
# M2 aggregates the squared distance from the mean
# count aggregates the number of samples seen so far
def update(existingAggregate, newValue):
    (count, mean, M2) = existingAggregate
    count += 1
    delta = newValue - mean
    mean += delta / count
    delta2 = newValue - mean
    M2 += delta * delta2
    return (count, mean, M2)

# Retrieve the mean, variance and sample variance from an aggregate
def finalize(existingAggregate):
    (count, mean, M2) = existingAggregate
    if count < 2:
        return float("nan")
    else:
        (mean, variance, sampleVariance) = (mean, M2 / count, M2 / (count - 1))
        return (mean, variance, sampleVariance)

### Travel Startup: 
#### After graduating from USF, you found a startup that aims to provide personalized travel itineraries using big data analysis. Given your own personal preferences, build a plan for a year of travel across 5 locations. Or, in other words: pick 5 regions. What is the best time of year to visit them based on the dataset?
* You have to convince your potential customers that your travel itinerary is better than something they could come up with themselves with a little Googling. You can use pictures, information about local points of interest, etc.

Geohashes: *dk2* (The Bahamas), *9g3* (Mexico City), *c20* (Portland), *9xc* (Yellowstone), *9q9* (Clear Lake)

In [50]:
df.createOrReplaceTempView("locations")

lake = spark.sql("SELECT * from locations WHERE (geohash LIKE '9qb%')")

lake.count()


51238

In [45]:
portland = spark.sql("SELECT * from locations WHERE (geohash LIKE 'c20%')")

portland.count()

49368

In [46]:
bahamas = spark.sql("SELECT * from locations WHERE (geohash LIKE 'dk2%')")

bahamas.count()

56474

In [47]:
mexico = spark.sql("SELECT * from locations WHERE (geohash LIKE '9g3%')")

mexico.count()

58718

In [49]:
yellowstone = spark.sql("SELECT * from locations WHERE (geohash LIKE '9xc%')")

yellowstone.count()

50116

In [51]:
yellowstone.take(20)

[Row(time=1554908400000, lat=44.25292921362976, lon=-110.96990375305577, albedo_surface=14.8, precipitable_water_entire_atmosphere_single_layer=5.976626, pressure_maximum_wind=23726.81, pressure_surface=76160.79, pressure_tropopause=26998.84, relative_humidity_zerodegc_isotherm=73.0, snow_depth_surface=0.04912, temperature_surface=274.11514, temperature_tropopause=216.91328, total_cloud_cover_entire_atmosphere_single_layer=100.0, total_precipitation_surface_3_hour_accumulation=0.0625, vegetation_surface=21.0, visibility_surface=24100.0, wilting_point_surface=0.066, wind_speed_gust_surface=13.020082, _c18=None, geohash='9xc5dzfpvzfm'),
 Row(time=1554908400000, lat=44.39119211262595, lon=-110.55727792459314, albedo_surface=15.3, precipitable_water_entire_atmosphere_single_layer=5.6766257, pressure_maximum_wind=22240.41, pressure_surface=74589.586, pressure_tropopause=26458.041, relative_humidity_zerodegc_isotherm=78.0, snow_depth_surface=0.04576, temperature_surface=273.30515, temperatur

### Clear Lake

Ideal Temperature: 65-75 F

Wind Speed: 0-5mph

Cloud Cover: 40-70%

**Ideal time: Jun-Aug**

In [71]:
jan = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201901*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

feb = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201902*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

mar = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201903*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

apr = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201904*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

may = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201905*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

jun = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201906*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

In [72]:
jul = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201907*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

aug = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201908*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

sep = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201909*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

octo = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201910*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

nov = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201911*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

dec = spark.read.load('hdfs://orion01:25001/2019/namanl_218_201912*.gz',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

In [73]:
jan = jan.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
jan = jan.withColumn("geohash", geohashEncodeUDF(jan.lat, jan.lon)).withColumn("temperature_surface",\
                                                                               k2f(jan.temperature_surface))
jan = jan.withColumn("wind_speed_gust_surface", jan.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

feb = feb.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
feb = feb.withColumn("geohash", geohashEncodeUDF(feb.lat, feb.lon)).withColumn("temperature_surface",\
                                                                               k2f(feb.temperature_surface))
feb = feb.withColumn("wind_speed_gust_surface", feb.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

mar = mar.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
mar = mar.withColumn("geohash", geohashEncodeUDF(mar.lat, mar.lon)).withColumn("temperature_surface",\
                                                                               k2f(mar.temperature_surface))
mar = mar.withColumn("wind_speed_gust_surface", mar.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

apr = apr.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
apr = apr.withColumn("geohash", geohashEncodeUDF(apr.lat, apr.lon)).withColumn("temperature_surface",\
                                                                               k2f(apr.temperature_surface))
apr = apr.withColumn("wind_speed_gust_surface", apr.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

may = may.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
may = may.withColumn("geohash", geohashEncodeUDF(may.lat, may.lon)).withColumn("temperature_surface",\
                                                                               k2f(may.temperature_surface))
may = may.withColumn("wind_speed_gust_surface", may.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

jun = jun.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
jun = jun.withColumn("geohash", geohashEncodeUDF(jun.lat, jun.lon)).withColumn("temperature_surface",\
                                                                               k2f(jun.temperature_surface))
jun = jun.withColumn("wind_speed_gust_surface", jun.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

jul = jul.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
jul = jul.withColumn("geohash", geohashEncodeUDF(jul.lat, jul.lon)).withColumn("temperature_surface",\
                                                                               k2f(jul.temperature_surface))
jul = jul.withColumn("wind_speed_gust_surface", jul.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

aug = aug.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
aug = aug.withColumn("geohash", geohashEncodeUDF(aug.lat, aug.lon)).withColumn("temperature_surface",\
                                                                               k2f(aug.temperature_surface))
aug = aug.withColumn("wind_speed_gust_surface", aug.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

sep = sep.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
sep = sep.withColumn("geohash", geohashEncodeUDF(sep.lat, sep.lon)).withColumn("temperature_surface",\
                                                                               k2f(sep.temperature_surface))
sep = sep.withColumn("wind_speed_gust_surface", sep.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

octo = octo.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
octo = octo.withColumn("geohash", geohashEncodeUDF(octo.lat, octo.lon)).withColumn("temperature_surface",\
                                                                               k2f(octo.temperature_surface))
octo = octo.withColumn("wind_speed_gust_surface", octo.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

nov = nov.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
nov = nov.withColumn("geohash", geohashEncodeUDF(nov.lat, nov.lon)).withColumn("temperature_surface",\
                                                                               k2f(nov.temperature_surface))
nov = nov.withColumn("wind_speed_gust_surface", nov.wind_speed_gust_surface * 2.23694) # convert gust speed to mph

dec = dec.withColumnRenamed("1_time", "time").withColumnRenamed("2_lat", "lat").withColumnRenamed("3_lon", "lon")
dec = dec.withColumn("geohash", geohashEncodeUDF(dec.lat, dec.lon)).withColumn("temperature_surface",\
                                                                               k2f(dec.temperature_surface))
dec = dec.withColumn("wind_speed_gust_surface", dec.wind_speed_gust_surface * 2.23694) # convert gust speed to mph
    
jan.take(1)

[Row(time=1546722000000, lat=58.424226538128224, lon=-88.11817117586963, albedo_surface=65.0, precipitable_water_entire_atmosphere_single_layer=1.4045608, pressure_maximum_wind=11128.6875, pressure_surface=101525.72, pressure_tropopause=33332.16, relative_humidity_zerodegc_isotherm=74.0, snow_depth_surface=0.05, temperature_surface=-22.15636599999999, temperature_tropopause=218.16837, total_cloud_cover_entire_atmosphere_single_layer=0.0, total_precipitation_surface_3_hour_accumulation=0.0, vegetation_surface=0.0, visibility_surface=24100.0, wilting_point_surface=0.0, wind_speed_gust_surface=19.32594917852, _c18=None, geohash='f43k6gf3cc1q')]

In [74]:
jan.createOrReplaceTempView("jan")
feb.createOrReplaceTempView("feb")
mar.createOrReplaceTempView("mar")
apr.createOrReplaceTempView("apr")
may.createOrReplaceTempView("may")
jun.createOrReplaceTempView("jun")
jul.createOrReplaceTempView("jul")
aug.createOrReplaceTempView("aug")
sep.createOrReplaceTempView("sep")
octo.createOrReplaceTempView("oct")
nov.createOrReplaceTempView("nov")
dec.createOrReplaceTempView("dec")

### Clear Lake: Water Sports

Ideal Temperature: 65-75 F

Wind Speed: 0-5mph

Cloud Cover: 0-50%

**Ideal Time: Apr-Jul**

In [167]:
lake = spark.sql("SELECT * from jan WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


212

In [168]:
lake = spark.sql("SELECT * from feb WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


0

In [169]:
lake = spark.sql("SELECT * from mar WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


0

In [170]:
lake = spark.sql("SELECT * from may WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


8

In [171]:
lake = spark.sql("SELECT * from jun WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


778

In [172]:
lake = spark.sql("SELECT * from jul WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


840

In [173]:
lake = spark.sql("SELECT * from aug WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


986

In [174]:
lake = spark.sql("SELECT * from sep WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


328

In [175]:
lake = spark.sql("SELECT * from oct WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


0

In [176]:
lake = spark.sql("SELECT * from nov WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


0

In [88]:
lake = spark.sql("SELECT * from nov WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 40 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 70")

lake.count()

0

In [177]:
lake = spark.sql("SELECT * from dec WHERE (geohash LIKE '9qb%')\
                    AND 65 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 > total_cloud_cover_entire_atmosphere_single_layer")

lake.count()


0

### Bahamas: Sunbathing

Ideal Temperature: 75+ F

Wind Speed: 0-5mph

Cloud Cover: 0-10%

**Ideal Time: Jun-Aug**

In [178]:
bahamas = spark.sql("SELECT * from jan WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


36

In [179]:
bahamas = spark.sql("SELECT * from feb WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


16

In [180]:
bahamas = spark.sql("SELECT * from mar WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


22

In [181]:
bahamas = spark.sql("SELECT * from apr WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


43

In [182]:
bahamas = spark.sql("SELECT * from may WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


24

In [183]:
bahamas = spark.sql("SELECT * from jun WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


95

In [184]:
bahamas = spark.sql("SELECT * from jul WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


84

In [185]:
bahamas = spark.sql("SELECT * from aug WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


66

In [186]:
bahamas = spark.sql("SELECT * from sep WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


26

In [187]:
bahamas = spark.sql("SELECT * from oct WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


30

In [188]:
bahamas = spark.sql("SELECT * from nov WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


15

In [189]:
bahamas = spark.sql("SELECT * from dec WHERE (geohash LIKE 'dk2%')\
                    AND 75 < temperature_surface \
                    AND 5 >= wind_speed_gust_surface \
                    AND 0 < total_cloud_cover_entire_atmosphere_single_layer AND total_cloud_cover_entire_atmosphere_single_layer < 10")

bahamas.count()


0

### Yellowstone National Park: Rock Climbing

Ideal Temperature: 55-75 F

Wind Speed: 0-5mph

Cloud Cover: 50-100%

**Ideal Time: Jul-Aug**

In [117]:
yellowstone = spark.sql("SELECT * from jan WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


0

In [116]:
yellowstone = spark.sql("SELECT * from feb WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


0

In [118]:
yellowstone = spark.sql("SELECT * from mar WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


0

In [119]:
yellowstone = spark.sql("SELECT * from apr WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


0

In [120]:
yellowstone = spark.sql("SELECT * from may WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


1

In [121]:
yellowstone = spark.sql("SELECT * from jun WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


33

In [122]:
yellowstone = spark.sql("SELECT * from jul WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


115

In [123]:
yellowstone = spark.sql("SELECT * from aug WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


81

In [124]:
yellowstone = spark.sql("SELECT * from sep WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


27

In [125]:
yellowstone = spark.sql("SELECT * from oct WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


0

In [126]:
yellowstone = spark.sql("SELECT * from nov WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


0

In [127]:
yellowstone = spark.sql("SELECT * from dec WHERE (geohash LIKE '9xc%')\
                    AND 55 < temperature_surface AND temperature_surface < 75\
                    AND 5 >= wind_speed_gust_surface \
                    AND 50 < total_cloud_cover_entire_atmosphere_single_layer")

yellowstone.count()


0

### Mexico City: Touring the City

Ideal Temperature: 60-85 F

Wind Speed: 0-15mph

Cloud Cover: 50-100%

**Ideal Time: Sept**

In [151]:
mexico = spark.sql("SELECT * from jan WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


1

In [150]:
mexico = spark.sql("SELECT * from feb WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


0

In [149]:
mexico = spark.sql("SELECT * from mar WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


0

In [148]:
mexico = spark.sql("SELECT * from apr WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


16

In [147]:
mexico = spark.sql("SELECT * from may WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


0

In [146]:
mexico = spark.sql("SELECT * from jun WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


10

In [145]:
mexico = spark.sql("SELECT * from jul WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


2

In [144]:
mexico = spark.sql("SELECT * from aug WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


3

In [143]:
mexico = spark.sql("SELECT * from sep WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


62

In [142]:
mexico = spark.sql("SELECT * from oct WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


14

In [141]:
mexico = spark.sql("SELECT * from nov WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


3

In [140]:
mexico = spark.sql("SELECT * from dec WHERE (geohash LIKE '9g3%')\
                    AND 60 <= temperature_surface AND temperature_surface <= 85\
                    AND 15 <= wind_speed_gust_surface \
                    AND 50 <= total_cloud_cover_entire_atmosphere_single_layer")

mexico.count()


1

### Portland: Hiking

Ideal Temperature: 50-70 F

Wind Speed: 0-10mph

Cloud Cover: 40-100%

**Ideal Time: May-Jun, Sept**

In [154]:
portland = spark.sql("SELECT * from jan WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


355

In [155]:
portland = spark.sql("SELECT * from feb WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


0

In [156]:
portland = spark.sql("SELECT * from mar WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


0

In [157]:
portland = spark.sql("SELECT * from apr WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


201

In [158]:
portland = spark.sql("SELECT * from may WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


809

In [159]:
portland = spark.sql("SELECT * from jun WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


718

In [160]:
portland = spark.sql("SELECT * from jul WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


467

In [161]:
portland = spark.sql("SELECT * from aug WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


358

In [162]:
portland = spark.sql("SELECT * from sep WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


942

In [163]:
portland = spark.sql("SELECT * from oct WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


268

In [164]:
portland = spark.sql("SELECT * from nov WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


1

In [165]:
portland = spark.sql("SELECT * from dec WHERE (geohash LIKE 'c20%')\
                    AND 50 <= temperature_surface AND temperature_surface <= 70\
                    AND 10 <= wind_speed_gust_surface \
                    AND 40 <= total_cloud_cover_entire_atmosphere_single_layer")

portland.count()


49